### Imports

In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install path.py
!pip install pytorch3d
clear_output()

In [ ]:
import numpy as np
import math
import random
import os
import plotly.graph_objects as go
import plotly.express as px

import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, utils

from path import Path

random.seed = 42

In [ ]:
!wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
!unzip -q ModelNet10.zip

path = Path("ModelNet10")

folders = [dir for dir in sorted(os.listdir(path)) if os.path.isdir(path/dir)]

clear_output()
classes = {folder: i for i, folder in enumerate(folders)}
classes

{'bathtub': 0,
 'bed': 1,
 'chair': 2,
 'desk': 3,
 'dresser': 4,
 'monitor': 5,
 'night_stand': 6,
 'sofa': 7,
 'table': 8,
 'toilet': 9}

In [ ]:
def default_transforms():
    return transforms.Compose([
                               PointSampler(1024),
                               Normalize(),
                               RandomNoise(),
                               ToSorted(),
                               ToTensor()
    ])

!gdown https://drive.google.com/uc?id=1CVwVxdfUfP6TRcVUjjJvQeRcgCGcnSO_
from helping import *
clear_output()

### Data Preprocessing (optional)

In [ ]:
with open(path/"dresser/train/dresser_0001.off", 'r') as f:
    verts, faces = read_off(f)

i, j, k = np.array(faces).T
x, y, z = np.array(verts).T

# len(x)

In [ ]:
# visualize_rotate([go.Mesh3d(x=x, y=y, z=z, color='lightpink', opacity=0.50, i=i,j=j,k=k)]).show()
# visualize_rotate([go.Scatter3d(x=x, y=y, z=z, mode='markers')]).show()

In [ ]:
# pcshow(x, y, z)

In [ ]:
pointcloud = PointSampler(1024)((verts, faces))
# pcshow(*pointcloud.T)

norm_pointcloud = Normalize()(pointcloud)
# pcshow(*norm_pointcloud.T)

noisy_pointcloud = RandomNoise()(norm_pointcloud)
# pcshow(*noisy_pointcloud.T)

rot_pointcloud = RandomRotation_z()(noisy_pointcloud)
# pcshow(*rot_pointcloud.T)

sorted_pointcloud = ToSorted()(rot_pointcloud)
# pcshow(*sorted_pointcloud.T)

tensor_pointcloud = ToTensor()(sorted_pointcloud)

### Creating Loaders for Final Progress Report

#### Redefine classes

In [ ]:
class PointCloudData(Dataset):
    def __init__(self, root_dir, valid=False, folder="train", transform=default_transforms(), folders=None):
        self.root_dir = root_dir
        if not folders:
            folders = [dir for dir in sorted(os.listdir(root_dir)) if os.path.isdir(root_dir/dir)]
        self.classes = {folder: i for i, folder in enumerate(folders)}
        self.transforms = transform
        self.valid = valid
        self.pcs = []
        for category in self.classes.keys():
            new_dir = root_dir/Path(category)/folder
            for file in os.listdir(new_dir):
                if file.endswith('.off'):
                    sample = {}
                    with open(new_dir/file, 'r') as f:
                        verts, faces = read_off(f)
                    sample['pc'] = (verts, faces)
                    sample['category'] = category
                    self.pcs.append(sample)

    def __len__(self):
        return len(self.pcs)

    def __getitem__(self, idx):
        pointcloud = self.transforms(self.pcs[idx]['pc'])
        category = self.pcs[idx]['category']
        return pointcloud, self.classes[category]
    
class PointCloudDataPre(Dataset):
    def __init__(self, root_dir, valid=False, folder="train", transform=default_transforms(), folders=None):
        self.root_dir = root_dir
        if not folders:
            folders = [dir for dir in sorted(os.listdir(root_dir)) if os.path.isdir(root_dir/dir)]
        self.classes = {folder: i for i, folder in enumerate(folders)}
        self.transforms = transform
        self.valid = valid
        self.pcs = []
        for category in self.classes.keys():
            new_dir = root_dir/Path(category)/folder
            for file in os.listdir(new_dir):
                if file.endswith('.off'):
                    sample = {}
                    with open(new_dir/file, 'r') as f:
                        verts, faces = read_off(f)
                    sample['pc'] = self.transforms((verts, faces))
                    sample['category'] = category
                    self.pcs.append(sample)

    def __len__(self):
        return len(self.pcs)

    def __getitem__(self, idx):
        pointcloud = self.pcs[idx]['pc']
        category = self.pcs[idx]['category']
        return pointcloud, self.classes[category]
    
    
class PointCloudDataBoth(Dataset):
    def __init__(self, root_dir, valid=False, folder="train", static_transform=default_transforms(), later_transform=None, folders=None):
        self.root_dir = root_dir
        if not folders:
            folders = [dir for dir in sorted(os.listdir(root_dir)) if os.path.isdir(root_dir/dir)]
        self.classes = {folder: i for i, folder in enumerate(folders)}
        self.static_transform = static_transform
        self.later_transform = later_transform
        self.valid = valid
        self.pcs = []
        for category in self.classes.keys():
            new_dir = root_dir/Path(category)/folder
            for file in os.listdir(new_dir):
                if file.endswith('.off'):
                    sample = {}
                    with open(new_dir/file, 'r') as f:
                        verts, faces = read_off(f)
                    sample['pc'] = self.static_transform((verts, faces))
                    sample['category'] = category
                    self.pcs.append(sample)

    def __len__(self):
        return len(self.pcs)

    def __getitem__(self, idx):
        pointcloud = self.pcs[idx]['pc']
        if self.later_transform is not None:
            pointcloud = self.later_transform(pointcloud)
        category = self.pcs[idx]['category']
        return pointcloud, self.classes[category]

In [ ]:
!mkdir drive/MyDrive/Thesis/dataloaders/final

#### Overfitting - all augmentations applied before training

In [ ]:
BATCH_SIZE = 48

trs = transforms.Compose([
                        PointSampler(1024),
                        ToSorted(),
                        Normalize(),
                        ToTensor()
])

beds_train_dataset = PointCloudDataPre(path, folders=['bed'], transform=trs)
beds_valid_dataset = PointCloudDataPre(path, folder='test', folders=['bed'], transform=trs)

beds_train_loader = DataLoader(dataset=beds_train_dataset, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
beds_valid_loader = DataLoader(dataset=beds_valid_dataset, batch_size=BATCH_SIZE, drop_last=True)

!mkdir dataloader_beds_pre
torch.save(beds_train_loader, 'dataloader_beds_pre/trainloader.pth')
torch.save(beds_valid_loader, 'dataloader_beds_pre/validloader.pth')

!mkdir drive/MyDrive/Thesis/dataloaders/final
!cp -r dataloader_beds_pre drive/MyDrive/Thesis/dataloaders/final

mkdir: cannot create directory ‘dataloader_beds_pre’: File exists
mkdir: cannot create directory ‘drive/MyDrive/Thesis/dataloaders/final’: File exists


#### Underfitting - all augmentations applied during training

In [ ]:
BATCH_SIZE = 48

trs = transforms.Compose([
                               PointSampler(1024),
                               ToSorted(),
                               Normalize(),
                               RandomNoise(),
                               ToTensor()
])

beds_train_dataset = PointCloudData(path, folders=['bed'], transform=trs)
beds_valid_dataset = PointCloudData(path, folder='test', folders=['bed'], transform=trs)

beds_train_loader = DataLoader(dataset=beds_train_dataset, num_workers=4, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
beds_valid_loader = DataLoader(dataset=beds_valid_dataset, num_workers=4, batch_size=BATCH_SIZE, drop_last=True)

!mkdir dataloader_beds_dur
torch.save(beds_train_loader, 'dataloader_beds_dur/trainloader.pth')
torch.save(beds_valid_loader, 'dataloader_beds_dur/validloader.pth')

!cp -r dataloader_beds_dur drive/MyDrive/Thesis/dataloaders/final

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



mkdir: cannot create directory ‘dataloader_beds_dur’: File exists


#### Both - static and dynamic transformations

In [ ]:
BATCH_SIZE = 48

static_trs = transforms.Compose([
                               PointSampler(1024),
                               ToSorted(),
                               Normalize(),
])

dynamic_trs = transforms.Compose([
                               RandomNoise(),
                               ToTensor()
])

beds_train_dataset = PointCloudDataBoth(path, folders=['bed'], static_transform=static_trs, later_transform=dynamic_trs)
beds_valid_dataset = PointCloudDataBoth(path, folder='test', folders=['bed'], static_transform=static_trs)

beds_train_loader = DataLoader(dataset=beds_train_dataset, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
beds_valid_loader = DataLoader(dataset=beds_valid_dataset, batch_size=BATCH_SIZE, drop_last=True)

!mkdir dataloader_beds_both
torch.save(beds_train_loader, 'dataloader_beds_both/trainloader.pth')
torch.save(beds_valid_loader, 'dataloader_beds_both/validloader.pth')

!cp -r dataloader_beds_both drive/MyDrive/Thesis/dataloaders/final

mkdir: cannot create directory ‘dataloader_beds_both’: File exists


#### Two classes: beds and tables

In [ ]:
BATCH_SIZE = 48

static_trs = transforms.Compose([
                               PointSampler(1024),
                               ToSorted(),
                               Normalize(),
])

dynamic_trs = transforms.Compose([
                               RandomNoise(),
                               ToTensor()
])

beds_train_dataset = PointCloudDataBoth(path, folders=['bed', 'table'], static_transform=static_trs, later_transform=dynamic_trs)
beds_valid_dataset = PointCloudDataBoth(path, folder='test', folders=['bed', 'table'], static_transform=trs)

beds_train_loader = DataLoader(dataset=beds_train_dataset, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
beds_valid_loader = DataLoader(dataset=beds_valid_dataset, batch_size=BATCH_SIZE, drop_last=True)

!mkdir dataloader_beds_tables
torch.save(beds_train_loader, 'dataloader_beds_tables/trainloader.pth')
torch.save(beds_valid_loader, 'dataloader_beds_tables/validloader.pth')

!cp -r dataloader_beds_tables drive/MyDrive/Thesis/dataloaders/final

### For 512

In [ ]:
!mkdir drive/MyDrive/Thesis/dataloaders/final512

#### Overfitting - all augmentations applied before training

In [ ]:
BATCH_SIZE = 48

trs = transforms.Compose([
                        PointSampler(512),
                        ToSorted(),
                        Normalize(),
                        ToTensor()
])

beds_train_dataset = PointCloudDataPre(path, folders=['bed'], transform=trs)
beds_valid_dataset = PointCloudDataPre(path, folder='test', folders=['bed'], transform=trs)

beds_train_loader = DataLoader(dataset=beds_train_dataset, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
beds_valid_loader = DataLoader(dataset=beds_valid_dataset, batch_size=BATCH_SIZE, drop_last=True)

!mkdir dataloader_beds_pre
torch.save(beds_train_loader, 'dataloader_beds_pre/trainloader.pth')
torch.save(beds_valid_loader, 'dataloader_beds_pre/validloader.pth')

!mkdir drive/MyDrive/Thesis/dataloaders/final
!cp -r dataloader_beds_pre drive/MyDrive/Thesis/dataloaders/final512

mkdir: cannot create directory ‘dataloader_beds_pre’: File exists
mkdir: cannot create directory ‘drive/MyDrive/Thesis/dataloaders/final’: File exists


#### Underfitting - all augmentations applied during training

In [ ]:
BATCH_SIZE = 48

trs = transforms.Compose([
                               PointSampler(512),
                               ToSorted(),
                               Normalize(),
                               ToTensor()
])

beds_train_dataset = PointCloudData(path, folders=['bed'], transform=trs)
beds_valid_dataset = PointCloudData(path, folder='test', folders=['bed'], transform=trs)

beds_train_loader = DataLoader(dataset=beds_train_dataset, num_workers=4, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
beds_valid_loader = DataLoader(dataset=beds_valid_dataset, num_workers=4, batch_size=BATCH_SIZE, drop_last=True)

!mkdir dataloader_beds_dur
torch.save(beds_train_loader, 'dataloader_beds_dur/trainloader.pth')
torch.save(beds_valid_loader, 'dataloader_beds_dur/validloader.pth')

!cp -r dataloader_beds_dur drive/MyDrive/Thesis/dataloaders/final512

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



mkdir: cannot create directory ‘dataloader_beds_dur’: File exists


#### Both - static and dynamic transformations

In [ ]:
BATCH_SIZE = 48

static_trs = transforms.Compose([
                               PointSampler(512),
                               ToSorted(),
                               Normalize(),
])

dynamic_trs = transforms.Compose([
                               RandomNoise(),
                               ToTensor()
])

beds_train_dataset = PointCloudDataBoth(path, folders=['bed'], static_transform=static_trs, later_transform=dynamic_trs)
beds_valid_dataset = PointCloudDataBoth(path, folder='test', folders=['bed'], static_transform=static_trs)

beds_train_loader = DataLoader(dataset=beds_train_dataset, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
beds_valid_loader = DataLoader(dataset=beds_valid_dataset, batch_size=BATCH_SIZE, drop_last=True)

!mkdir dataloader_beds_both
torch.save(beds_train_loader, 'dataloader_beds_both/trainloader.pth')
torch.save(beds_valid_loader, 'dataloader_beds_both/validloader.pth')

!cp -r dataloader_beds_both drive/MyDrive/Thesis/dataloaders/final512

mkdir: cannot create directory ‘dataloader_beds_both’: File exists


#### Two classes: beds and tables

In [ ]:
BATCH_SIZE = 48

static_trs = transforms.Compose([
                               PointSampler(512),
                               ToSorted(),
                               Normalize(),
])

dynamic_trs = transforms.Compose([
                               RandomNoise(),
                               ToTensor()
])

beds_train_dataset = PointCloudDataBoth(path, folders=['bed', 'table'], static_transform=static_trs, later_transform=dynamic_trs)
beds_valid_dataset = PointCloudDataBoth(path, folder='test', folders=['bed', 'table'], static_transform=trs)

beds_train_loader = DataLoader(dataset=beds_train_dataset, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
beds_valid_loader = DataLoader(dataset=beds_valid_dataset, batch_size=BATCH_SIZE, drop_last=True)

!mkdir dataloader_beds_tables
torch.save(beds_train_loader, 'dataloader_beds_tables/trainloader.pth')
torch.save(beds_valid_loader, 'dataloader_beds_tables/validloader.pth')

!cp -r dataloader_beds_tables drive/MyDrive/Thesis/dataloaders/final